# Solving Full Cell Simulation Using the Discontinuous Galerkin Method with Interior Penalty
Author: E Leshinka Molel

In this notebook we set to solve the full cell simulation equations for a solid-state battery. The full-cell is simplified to include three domains
- negative electrode
- solid electrolyte (SE) separator
- positive electrode

At the interface between negative electrode and SE, and the interface between SE and positive electrode, we have charge transfer reactions. For simplicity, linear kinetics are assumed.

Phase 1 is the solid active material and phase 2 is the SE.

Current flowing due to charge transfer reaction is given by linear kinetics expression:
$$i = \frac{F i_{o}}{RT}(\phi_1 - \phi_2 - U)$$

The above expression can be written twice, once for -ve electrode and once for +ve electrode.

In our case, for simplicity, infinite kinetics are assumed at the negative electrode and SE separator.

## Setting Up Variational Formulation
$$\nabla \cdot (-\kappa \nabla u) = f$$
$$u(x=0,y) = u_0$$
$$u(x=L_x,y) = u_{L_x}$$
$$-\kappa \nabla u \cdot \hat{n}|_{y=0,y=L_y} = 0$$

The total domain is indicated by $\Omega$ and the external boundary of the domain by $\partial \Omega$.

Because of the internal discontinuity in $u$, we use Discontinuous Lagrange elements. We solve the partial differential equation for each element $K_i$ then sum over all the $N$ elements. This involves multiplying by a test function $v$ and integrating over the element $K_i$.

$$\sum_i^N \int_{K_i} -\nabla \cdot(\kappa u_i) v \mathrm{dx} = \sum_i^N \int_{K_i} f_i v_i \mathrm{dx}$$
Integrating the LHS by parts, we obtain:
$$\sum_i^N \int_{K_i} -\kappa \nabla u_i \cdot \nabla v_i \mathrm{dx} + \sum_i^N \int_{\partial K_i} \kappa \hat{n_i} \cdot \nabla u_i \cdot v_i \mathrm{ds} = \sum_i^N \int_{K_i} f_i v_i \mathrm{dx}$$

We can split the second term on the LHS to have the parts of the external boundary: $\bigcup \partial \Omega_i = \partial \Omega$ for $i=1,...,N$. Also $\partial \Omega_i \bigcup \partial \hat{K_i} = \partial K_i$.

Therefore, we have:
$$\int_{\Omega} -\kappa \nabla u \cdot \nabla v \mathrm{dx} + \int_{\partial \Omega} \kappa \hat{n} \cdot \nabla u \cdot v \mathrm{ds} + \sum_i^N \int_{\partial \hat{K_i}} \kappa \hat{n_i} \cdot \nabla u_i \cdot v_i \mathrm{ds} = \int_{\Omega} f v \mathrm{dx}$$

The second term on the LHS can be used to set the Neumann boundary conditions where we have those. Below, we expound further on the third term on the LHS.

We note that for the third term, we have an integral for each side of the integral facet, say between cell $i$ and $j$. We can therefore write this integral as a jump integral (noting that $n_i=-n_j$) over the set of all internal facets $E_j, j=0,\dots,N_{internal}$

\begin{align}
\sum_i^N \int_{\partial \hat{K_i}} \kappa \hat{n_i} \cdot \nabla u_i \cdot v_i \mathrm{ds} = -\sum_j^{N_{internal}} \int_{E_j} \kappa \hat{n_j} \cdot [\nabla u_j \cdot v_j] \mathrm{ds}
\end{align}

We can further expand the RHS of above using the relation $[a \cdot b] = \langle a\rangle \cdot [b] + [a] \cdot \langle b \rangle$. Where $\langle \text{value} \rangle$ denotes average of the value across the shared internal boundary. Therefore, we can rewrite our expression above into:

\begin{align}
\sum_i^N \int_{\partial \hat{K_i}} \kappa \hat{n_i} \cdot \nabla u_i \cdot v_i \mathrm{ds} = -\sum_j^{N_{internal}} \int_{E_j} \kappa \hat{n_j} \cdot [\nabla u_j] \cdot \langle v_j\rangle \mathrm{ds} -\sum_j^{N_{internal}} \int_{E_j} \kappa \hat{n_j} \cdot \langle \nabla u_j\rangle \cdot [v_j] \mathrm{ds}
\end{align}

Because we want our solution to be conservative, we enforce that the jump of the gradient in normal direction is zero by removing the term involving $[\nabla u]$ from the RHS of the above expression.

To maintain symmetry when $u$ and $v$ are switched, we add a term $-\sum_j^{N_{internal}} \int_{E_j} \kappa \hat{n_j} \cdot \langle \nabla v_j\rangle \cdot [u_j] \mathrm{ds}$ to the RHS. We also add another term to the RHS for coercivity: $\int_{E_j}\frac{\gamma}{\langle h \rangle}[u][v]~\mathrm{d}s$ where $h$ is the diameter of the circumscribed circle.

We use Nitsche's method to impose Dirichlet and Neumann boundary conditions on the exterior boundary $\partial \Omega$.

In [1]:
import os

import gmsh
import meshio
import numpy as np
import pyvista
import ufl
import warnings

from dolfinx import cpp, fem, io, mesh, nls, plot
from mpi4py import MPI
from petsc4py import PETSc
from ufl import (Circumradius, FacetNormal, SpatialCoordinate, TrialFunction, TestFunction,
                 div, dx, ds, dS, grad, inner, grad, avg, jump)

import commons, geometry, utils

warnings.simplefilter('ignore')

In [2]:
adaptive_refine = False
markers = {
    'ne_pcc': 0,  # boundary not included yet
    'ne_se': 1,  # left boundary
    'pe_se': 2,  # internal boundary
    'pe_pcc': 3,  # left boundary
    'ne': 4,   # domain not included yet
    'se': 5,  # left domain
    'pe': 6,  # right domain
    "insulated": 7,  # top and bottom boundary
}

points = [
    (5, 0, 0),
    (10, 0, 0),
    (20, 0, 0),
    (20, 5, 0),
    (10, 5, 0),
    (5, 5, 0),
]
gpoints = []
lines = []
workdir = "output/full-cell"
utils.make_dir_if_missing(workdir)
output_meshfile = os.path.join(workdir, 'mesh.msh')
tria_meshfile = os.path.join(workdir, "tria.xdmf")
line_meshfile = os.path.join(workdir, "line.xdmf")
potential_resultsfile = os.path.join(workdir, "potential.xdmf")
current_resultsfile = os.path.join(workdir, "current.xdmf")
gmsh.initialize()
gmsh.model.add('full-cell')
gmsh.option.setNumber("Mesh.CharacteristicLengthMin", 0.1)
gmsh.option.setNumber("Mesh.CharacteristicLengthMax", 0.1)

for idx, p in enumerate(points):
    gpoints.append(
        gmsh.model.occ.addPoint(*p)
    )
for idx in range(0, len(points)-1):
    lines.append(
        gmsh.model.occ.addLine(gpoints[idx], gpoints[idx+1])
    )
lines.append(
    gmsh.model.occ.addLine(gpoints[-1], gpoints[0])
)
lines.append(
    gmsh.model.occ.addLine(gpoints[4], gpoints[1])
)

gmsh.model.occ.synchronize()
gmsh.model.addPhysicalGroup(1, [lines[5]], markers['ne_se'])
gmsh.model.addPhysicalGroup(1, [lines[6]], markers['pe_se'])
gmsh.model.addPhysicalGroup(1, [lines[2]], markers['pe_pcc'])
gmsh.model.addPhysicalGroup(1, [lines[idx] for idx in [0, 1, 3, 4]], markers['insulated'])
gmsh.model.occ.synchronize()
se_loop = gmsh.model.occ.addCurveLoop([lines[idx] for idx in [0, 6, 4, 5]])
pe_loop = gmsh.model.occ.addCurveLoop([lines[idx] for idx in [1, 2, 3, 6]])
gmsh.model.occ.synchronize()
se_phase = gmsh.model.occ.addPlaneSurface([se_loop])
pe_phase = gmsh.model.occ.addPlaneSurface([pe_loop])
gmsh.model.occ.synchronize()
gmsh.model.addPhysicalGroup(2, [se_phase], markers['se'])
gmsh.model.addPhysicalGroup(2, [pe_phase], markers['pe'])
gmsh.model.occ.synchronize()

# adaptive refinement
if adaptive_refine:
    gmsh.model.mesh.field.add("Distance", 1)
    gmsh.model.mesh.field.setNumbers(1, "EdgesList", [lines[idx] for idx in [6]])
    
    gmsh.model.mesh.field.add("Threshold", 2)
    gmsh.model.mesh.field.setNumber(2, "IField", 1)
    gmsh.model.mesh.field.setNumber(2, "LcMin", 0.01)
    gmsh.model.mesh.field.setNumber(2, "LcMax", 1)
    gmsh.model.mesh.field.setNumber(2, "DistMin", 0)
    gmsh.model.mesh.field.setNumber(2, "DistMax", 0.5)
    
    gmsh.model.mesh.field.add("Max", 5)
    gmsh.model.mesh.field.setNumbers(5, "FieldsList", [2])
    gmsh.model.mesh.field.setAsBackgroundMesh(5)

gmsh.model.mesh.generate(2)
gmsh.write(output_meshfile)
gmsh.finalize()

mesh_2d = meshio.read(output_meshfile)
tria_mesh = geometry.create_mesh(mesh_2d, "triangle")
meshio.write(tria_meshfile, tria_mesh)
line_mesh = geometry.create_mesh(mesh_2d, "line")
meshio.write(line_meshfile, line_mesh)

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 20%] Meshing curve 2 (Line)
Info    : [ 30%] Meshing curve 3 (Line)
Info    : [ 50%] Meshing curve 4 (Line)
Info    : [ 60%] Meshing curve 5 (Line)
Info    : [ 80%] Meshing curve 6 (Line)
Info    : [ 90%] Meshing curve 7 (Line)
Info    : Done meshing 1D (Wall 0.000690256s, CPU 0.001035s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : [ 50%] Meshing surface 2 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.193996s, CPU 0.193739s)
Info    : 8931 nodes 17916 elements
Info    : Writing 'output/full-cell/mesh.msh'...
Info    : Done writing 'output/full-cell/mesh.msh'



In [3]:
comm = MPI.COMM_WORLD
with io.XDMFFile(comm, tria_meshfile, "r") as infile3:
    domain = infile3.read_mesh(cpp.mesh.GhostMode.none, 'Grid')
    ct = infile3.read_meshtags(domain, name="Grid")
domain.topology.create_connectivity(domain.topology.dim, domain.topology.dim - 1)
with io.XDMFFile(comm, line_meshfile, "r") as infile2:
    ft = infile2.read_meshtags(domain, name="Grid")
meshtags = mesh.meshtags(domain, domain.topology.dim - 1, ft.indices, ft.values)
domaintags = mesh.meshtags(domain, domain.topology.dim, ct.indices, ct.values)

In [4]:
ds = ufl.Measure("ds", domain=domain, subdomain_data=meshtags)
dS = ufl.Measure("dS", domain=domain, subdomain_data=meshtags)

In [5]:
V = fem.FunctionSpace(domain, ("DG", 1))
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)
n = ufl.FacetNormal(domain)
x = ufl.SpatialCoordinate(domain)

alpha = 10
gamma = 10

h = 2 * ufl.Circumradius(domain)
h_avg = avg(h)

In [6]:
x = SpatialCoordinate(domain)

f = fem.Constant(domain, PETSc.ScalarType(0))
g = fem.Constant(domain, PETSc.ScalarType(0))
voltage = 100e-3  # 100 mV
u_left = fem.Function(V)
with u_left.vector.localForm() as u0_loc:
    u0_loc.set(0)
u_right = fem.Function(V)
with u_right.vector.localForm() as u1_loc:
    u1_loc.set(voltage)

In [7]:
i0 = fem.Constant(domain, PETSc.ScalarType(10))
faraday_const = fem.Constant(domain, PETSc.ScalarType(96485))
R = fem.Constant(domain, PETSc.ScalarType(8.314))
T = fem.Constant(domain, PETSc.ScalarType(298))

U = fem.Function(V)
with U.vector.localForm() as U_loc:
    U_loc.set(1e-3)

In [8]:
a = inner(grad(u), grad(v)) * dx - inner(n, grad(u)) * v * ds

# Add DG/IP terms
a += - inner(avg(grad(v)), jump(u, n)) * dS - inner(jump(v, n), avg(grad(u))) * dS
a += + (gamma / h_avg) * inner(jump(v, n), jump(u, n)) * dS

# internal boundary
a +=  -(i0 * faraday_const / R / T) * inner(avg(grad(v)), jump(u, n)) * dS(markers['pe_se']) - inner(jump(v, n), avg(grad(u))) * dS(markers['pe_se'])

# Add Nitsche terms
a += - inner(n, grad(v)) * u * ds(markers['ne_se']) + alpha / h * inner(u, v) * ds(markers['ne_se'])
a += - inner(n, grad(v)) * u * ds(markers['pe_pcc']) + alpha / h * inner(u, v) * ds(markers['pe_pcc'])
L = inner(f, v) * dx 
L += - inner(n, grad(v)) * u_left * ds(markers['ne_se']) + alpha / h * inner(u_left, v) * ds(markers['ne_se'])
L += - inner(n, grad(v)) * u_right * ds(markers['pe_pcc']) + alpha / h * inner(u_right, v) * ds(markers['pe_pcc'])

# Neumann BC on insulated boundary
L += - (h / alpha) * inner(n, grad(v)) * g * ds(markers['insulated'])
L += + inner(g, v) * ds(markers['insulated'])

# L += (i0 * faraday_const / R / T) * inner(v, U) * ds(markers['pe_se'])
# L += - (h / alpha) * inner(n, grad(v)) * U * ds(markers['pe_se'])

In [9]:
problem = fem.petsc.LinearProblem(a, L)
uh = problem.solve()
uh.name = 'potential'

INFO:root:running build_ext
INFO:root:building 'libffcx_forms_b5d20a46f5a25be30a358a63236734eba45ef95b' extension
INFO:root:/home/molel/.conda/envs/fenicsx-env/bin/x86_64-conda-linux-gnu-cc -Wno-unused-result -Wsign-compare -DNDEBUG -fwrapv -O2 -Wall -fPIC -O2 -isystem /home/molel/.conda/envs/fenicsx-env/include -fPIC -O2 -isystem /home/molel/.conda/envs/fenicsx-env/include -march=nocona -mtune=haswell -ftree-vectorize -fPIC -fstack-protector-strong -fno-plt -O2 -ffunction-sections -pipe -isystem /home/molel/.conda/envs/fenicsx-env/include -DNDEBUG -D_FORTIFY_SOURCE=2 -O2 -isystem /home/molel/.conda/envs/fenicsx-env/include -fPIC -I/home/molel/.conda/envs/fenicsx-env/lib/python3.10/site-packages/ffcx/codegeneration -I/home/molel/.conda/envs/fenicsx-env/include/python3.10 -c libffcx_forms_b5d20a46f5a25be30a358a63236734eba45ef95b.c -o ./libffcx_forms_b5d20a46f5a25be30a358a63236734eba45ef95b.o -O2 -g0
INFO:root:/home/molel/.conda/envs/fenicsx-env/bin/x86_64-conda-linux-gnu-cc -shared -Wl,

In [10]:
with io.XDMFFile(comm, potential_resultsfile, "w") as outfile:
    outfile.write_mesh(domain)
    outfile.write_function(uh)

In [11]:
W = fem.VectorFunctionSpace(domain, ("DG", 0))
current_expr = fem.Expression(-grad(uh), W.element.interpolation_points())
current_h = fem.Function(W)
current_h.name = 'current density'
current_h.interpolate(current_expr)
I_left = fem.assemble_scalar(fem.form(inner(current_h, n) * ds(markers['ne_se'])))
I_right = fem.assemble_scalar(fem.form(inner(current_h, n) * ds(markers['pe_pcc'])))
print(f"out: {I_left:.2e} A/m2, in: {abs(I_right):.2e} A/m2")

INFO:root:running build_ext
INFO:root:building 'libffcx_forms_5c08671469c68adaf3b046f5e7e211e7af9ed02c' extension
INFO:root:/home/molel/.conda/envs/fenicsx-env/bin/x86_64-conda-linux-gnu-cc -Wno-unused-result -Wsign-compare -DNDEBUG -fwrapv -O2 -Wall -fPIC -O2 -isystem /home/molel/.conda/envs/fenicsx-env/include -fPIC -O2 -isystem /home/molel/.conda/envs/fenicsx-env/include -march=nocona -mtune=haswell -ftree-vectorize -fPIC -fstack-protector-strong -fno-plt -O2 -ffunction-sections -pipe -isystem /home/molel/.conda/envs/fenicsx-env/include -DNDEBUG -D_FORTIFY_SOURCE=2 -O2 -isystem /home/molel/.conda/envs/fenicsx-env/include -fPIC -I/home/molel/.conda/envs/fenicsx-env/lib/python3.10/site-packages/ffcx/codegeneration -I/home/molel/.conda/envs/fenicsx-env/include/python3.10 -c libffcx_forms_5c08671469c68adaf3b046f5e7e211e7af9ed02c.c -o ./libffcx_forms_5c08671469c68adaf3b046f5e7e211e7af9ed02c.o -O2 -g0
libffcx_forms_5c08671469c68adaf3b046f5e7e211e7af9ed02c.c: In function 'functionspace_for

VerificationError: CompileError: command '/home/molel/.conda/envs/fenicsx-env/bin/x86_64-conda-linux-gnu-cc' failed with exit code 1

In [ ]:
with io.XDMFFile(comm, current_resultsfile, "w") as outfile:
    outfile.write_mesh(domain)
    outfile.write_function(current_h)

In [ ]:
pyvista.start_xvfb()
# We create a mesh consisting of the degrees of freedom for visualization
topology, cell_types, x = plot.create_vtk_mesh(V)
num_dofs_local = V.dofmap.index_map.size_local
grid = pyvista.UnstructuredGrid(topology, cell_types, V.tabulate_dof_coordinates()[:num_dofs_local])
# To make this function work in parallel, we only consider dofs owned by the current process
grid.point_data["u"] = uh.vector.array.real
grid.set_active_scalars("u")
warped = grid.warp_by_scalar("u", factor=1)

# Create plotter
plotter = pyvista.Plotter()
pyvista.start_xvfb(wait=0.05)
plotter = pyvista.Plotter()
plotter.add_mesh(warped, show_edges=True, show_scalar_bar=True)
plotter.view_xy()
if not pyvista.OFF_SCREEN:
    plotter.show()
else:
    figure = plotter.screenshot("DG.png")